# Cross-resonance gate between a transmon and a fluxonium: optimization

In [20]:
import numpy as np
import scipy.integrate
import time
import qutip as qtp
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pysqkit
from pysqkit import QubitSystem
from pysqkit.drives.pulse_shapes import gaussian_top
from pysqkit.util.metrics import average_process_fidelity, \
    average_gate_fidelity
from pysqkit.util.phys import temperature_to_thermalenergy
from pysqkit.util.quantum import generalized_rabi_frequency
import pysqkit.util.transformations as trf
from pysqkit.util.linalg import get_mat_elem
from pysqkit.solvers.solvkit import integrate
from pysqkit.util.hsbasis import pauli_by_index
from pysqkit.solvers import solvkit
from pysqkit.drives.pulse_shapes import gaussian_top
import qutip
from typing import List, Dict, Callable
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
import util_tf_cr
import copy
import json
import cmath

from IPython.display import display, Latex

In [21]:
def func_to_minimize(
    pulse_time: list,
    t_rise: float,
    cr_coeff: float
) -> float:
    step = 1e-3
    n_points = int(pulse_time[0]/step)
    times = np.linspace(0, pulse_time[0], n_points)
    pulse = gaussian_top(times, t_rise, pulse_time[0])
    integral = scipy.integrate.simpson(2*np.pi*cr_coeff*pulse, times)
    return np.abs(integral - np.pi/4)  

def cry(theta):
    ide = np.identity(4)
    yz = np.kron(np.array([[0, -1j], [1j, 0]]), np.array([[1, 0], [0, -1]]))
    return np.cos(theta/2)*ide - 1j*np.sin(theta/2)*yz

def crx(theta):
    ide = np.identity(4)
    zx = np.kron(np.array([[0, 1], [1, 0]]), np.array([[1, 0], [0, -1]]))
    return np.cos(theta/2)*ide - 1j*np.sin(theta/2)*zx

def ry_t(theta):
    rot_y = np.cos(theta/2)*np.identity(2) - 1j*np.sin(theta/2)*np.array([[0, -1j], [1j, 0]])
    return np.kron(rot_y, np.identity(2))

def ry_f(theta):
    rot_y = np.cos(theta/2)*np.identity(2) - 1j*np.sin(theta/2)*np.array([[0, -1j], [1j, 0]])
    return np.kron(np.identity(2), rot_y)

def optimal_sup_op(
    sup_op_target: np.ndarray,
    sup_op: np.ndarray,
    hs_basis: Callable[[int, int], np.ndarray] = pauli_by_index
):
    sq_corr = util_tf_cr.single_qubit_corrections(sup_op, hs_basis)
    sq_corr_sup = trf.kraus_to_super(sq_corr, hs_basis)
    total_sup_op = sq_corr_sup.dot(sup_op)
    fid_list_ry = []
    theta_list = list(np.linspace(0, 2*np.pi, 200))
    for theta in theta_list:
        rot_y_super = trf.kraus_to_super(ry_t(theta), hs_basis)
        fid_list_ry.append(average_process_fidelity(sup_op_target, rot_y_super.dot(total_sup_op)))

    fid_ry = np.array(fid_list_ry)

    max_index = np.argmax(fid_ry)
    sup_rot_y_opt = trf.kraus_to_super(ry_t(theta_list[max_index]), hs_basis)
    total_sup_op_ry = sup_rot_y_opt.dot(total_sup_op)
    
    return total_sup_op_ry, theta_list[max_index]

def get_corrected_sup_op(
    sup_op: np.ndarray,
    theta_y: float, 
    hs_basis: Callable[[int, int], np.ndarray] = pauli_by_index
) -> np.ndarray:
    sq_corr = util_tf_cr.single_qubit_corrections(sup_op, hs_basis)
    sq_corr_sup = trf.kraus_to_super(sq_corr, hs_basis)
    total_sup_op = sq_corr_sup.dot(sup_op)
    sup_rot_y_opt = trf.kraus_to_super(ry_t(theta_y), hs_basis)
    total_sup_op_ry = sup_rot_y_opt.dot(total_sup_op)
    return total_sup_op_ry
    
    

In [22]:
def get_initial_params() -> dict:
    
    with open('../flx_transm_params.txt') as param_file:
        parameters_set = json.load(param_file)
    temperature = 0.020 #0.020 # K
    thermal_energy = temperature_to_thermalenergy(temperature) # kb T/h in GHz
    d_comp = 4

    p_set = "3"


    #Transmon
    levels_t = 3
    transm = pysqkit.qubits.SimpleTransmon(
        label='T', 
        max_freq=parameters_set[p_set]["max_freq_t"], 
        anharm=parameters_set[p_set]["anharm_t"],
        diel_loss_tan=parameters_set[p_set]["diel_loss_tan_t"],
        env_thermal_energy=thermal_energy,    
        dim_hilbert=levels_t,
        dephasing_times=None 
    )

    #Fluxonium
    levels_f = 4

    flx = pysqkit.qubits.Fluxonium(
        label='F', 
        charge_energy=parameters_set[p_set]["charge_energy_f"], 
        induct_energy=parameters_set[p_set]["induct_energy_f"], 
        joseph_energy=parameters_set[p_set]["joseph_energy_f"], 
        diel_loss_tan=parameters_set[p_set]["diel_loss_tan_f"], 
        env_thermal_energy=thermal_energy,
        dephasing_times=None, 
        dim_hilbert=100
    )
    flx.diagonalize_basis(levels_f)

    # We also add a drive on the fluxonium
    flx.add_drive(
        pysqkit.drives.microwave_drive,
        label='cr_drive_f',
        pulse=pysqkit.drives.pulses.cos_modulation,
        pulse_shape=pysqkit.drives.pulse_shapes.gaussian_top
    )

    d_leak = levels_t*levels_f - d_comp

    jc = parameters_set[p_set]["jc"]
    coupled_sys = transm.couple_to(flx, coupling=pysqkit.couplers.capacitive_coupling, 
                                   strength=jc)
    

    states_label = coupled_sys.all_state_labels()
    states_dict = coupled_sys.states_as_dict(as_qobj=True)
    
    state_label = ["00", "01", "10", "11"]
    comp_states = {}
    for label in state_label:
        state_tmp = coupled_sys.state(label)[1]
        loc = np.argmax(np.abs(state_tmp))
        phase = cmath.phase(state_tmp[loc])
        state_tmp = np.exp(-1j*phase)*state_tmp
        comp_states[label] = state_tmp
    
    op = coupled_sys["F"].charge_op()
    freq_drive = transm.max_freq
    
    eps = 0.3
    
    t_rise = 10.0 # [ns]
    t_gate_0 = [200]
    cr_coeff = np.abs(util_tf_cr.mu_yz_flx(comp_states, op, eps))

    args_to_pass = (t_rise, cr_coeff) 


    start = time.time()

    minimization_result = minimize(func_to_minimize, t_gate_0, args=args_to_pass)

    end = time.time()

    t_gate = minimization_result['x'][0] #1/(util.y_z_flx(coupled_sys, 'F')*eps_drive*4)  # [ns]
    pts_per_drive_period = 10

    nb_points = int(t_gate*freq_drive*pts_per_drive_period)
    tlist = np.linspace(0, t_gate, nb_points)

    coupled_sys['F'].drives['cr_drive_f'].set_params(phase=0, time=tlist, rise_time=t_rise, pulse_time=t_gate,
                                                     amp=eps, freq=freq_drive)
    
    simu_opt = qtp.solver.Options()
    simu_opt.atol = 1e-12
    simu_opt.rtol = 1e-10
    
    my_hs_basis = pauli_by_index

    env_syst = pysqkit.tomography.TomoEnv(system=coupled_sys, time=2*np.pi*tlist, 
                                          options=simu_opt, with_noise=False)
    
    comp_states_list = []
    for key in comp_states.keys():
        comp_states_list.append(comp_states[key])
    
    avg_leakage = env_syst.leakage(comp_states_list)
    
    sup_op = env_syst.to_super(comp_states_list, my_hs_basis, n_process=4, speed_up=True)
    
    cr_super_target = trf.kraus_to_super(cry(-np.pi/2), my_hs_basis)
    
    opt_sup_op, theta_y = optimal_sup_op(cr_super_target, sup_op)
    
    f_gate = average_gate_fidelity(cr_super_target, opt_sup_op, avg_leakage)
    
    params = {}
    params["freq_drive"] = freq_drive
    params["eps"] = eps
    params["rise_time"] = t_rise
    params["gate_time"] = t_gate
    params["phase_drive"] = 0.0
    params["theta_y"] = theta_y
    params["f_gate"] = f_gate
    
    return params

In [23]:
start = time.time()

params_in = get_initial_params()

end = time.time()

print("Computation time = {} s ".format(end - start))

Computation time = 286.07391262054443 s 


In [25]:
params_in
params_in_list = []
params_in_list.append(params_in["gate_time"])
params_in_list.append(params_in["rise_time"])

In [26]:
params_in

{'freq_drive': 5.5,
 'eps': 0.3,
 'rise_time': 10.0,
 'gate_time': 301.68856998890726,
 'phase_drive': 0.0,
 'theta_y': 0.12629518205386103,
 'f_gate': 0.9876638688911411}

In [32]:
def get_infidelity(
    params: list
) -> dict:
    with open('../flx_transm_params.txt') as param_file:
        parameters_set = json.load(param_file)
    temperature = 0.020 #0.020 # K
    thermal_energy = temperature_to_thermalenergy(temperature) # kb T/h in GHz
    d_comp = 4
    
    eps = 0.3
    
    t_gate = params[0]
    t_rise = params[1]


    p_set = "3"


    #Transmon
    levels_t = 3
    transm = pysqkit.qubits.SimpleTransmon(
        label='T', 
        max_freq=parameters_set[p_set]["max_freq_t"], 
        anharm=parameters_set[p_set]["anharm_t"],
        diel_loss_tan=parameters_set[p_set]["diel_loss_tan_t"],
        env_thermal_energy=thermal_energy,    
        dim_hilbert=levels_t,
        dephasing_times=None 
    )
    
    freq_drive = transm.freq
    

    #Fluxonium
    levels_f = 4

    flx = pysqkit.qubits.Fluxonium(
        label='F', 
        charge_energy=parameters_set[p_set]["charge_energy_f"], 
        induct_energy=parameters_set[p_set]["induct_energy_f"], 
        joseph_energy=parameters_set[p_set]["joseph_energy_f"], 
        diel_loss_tan=parameters_set[p_set]["diel_loss_tan_f"], 
        env_thermal_energy=thermal_energy,
        dephasing_times=None, 
        dim_hilbert=100
    )
    flx.diagonalize_basis(levels_f)

    # We also add a drive on the fluxonium
    flx.add_drive(
        pysqkit.drives.microwave_drive,
        label='cr_drive_f',
        pulse=pysqkit.drives.pulses.cos_modulation,
        pulse_shape=pysqkit.drives.pulse_shapes.gaussian_top
    )

    d_leak = levels_t*levels_f - d_comp

    jc = parameters_set[p_set]["jc"]
    coupled_sys = transm.couple_to(flx, coupling=pysqkit.couplers.capacitive_coupling, strength=jc)
    

    states_label = coupled_sys.all_state_labels()
    states_dict = coupled_sys.states_as_dict(as_qobj=True)
    
    state_label = ["00", "01", "10", "11"]
    comp_states = {}
    for label in state_label:
        state_tmp = coupled_sys.state(label)[1]
        loc = np.argmax(np.abs(state_tmp))
        phase = cmath.phase(state_tmp[loc])
        state_tmp = np.exp(-1j*phase)*state_tmp
        comp_states[label] = state_tmp
    
    q_op = coupled_sys["F"].charge_op()
    
    pts_per_drive_period = 10

    nb_points = int(t_gate*freq_drive*pts_per_drive_period)
    tlist = np.linspace(0, t_gate, nb_points)

    coupled_sys['F'].drives['cr_drive_f'].set_params(phase=0.0, time=tlist, rise_time=t_rise, pulse_time=t_gate,
                                                     amp=eps, freq=freq_drive)
    
    
    simu_opt = qtp.solver.Options()
    simu_opt.atol = 1e-12
    simu_opt.rtol = 1e-10

    env_syst = pysqkit.tomography.TomoEnv(system=coupled_sys, time=2*np.pi*tlist, 
                                          options=simu_opt, with_noise=False)
    
    comp_states_list = []
    for key in comp_states.keys():
        comp_states_list.append(comp_states[key])
    
    my_hs_basis = pauli_by_index
    
    avg_leakage = env_syst.leakage(comp_states_list)
    
    sup_op = env_syst.to_super(comp_states_list, my_hs_basis, n_process=4, speed_up=True)
    
    cr_super_target = trf.kraus_to_super(cry(-np.pi/2), my_hs_basis)
    
    opt_sup_op, theta_y = optimal_sup_op(cr_super_target, sup_op)
    
    # opt_sup_op = get_corrected_sup_op(sup_op, theta_y)
    
    f_gate = average_gate_fidelity(cr_super_target, opt_sup_op, avg_leakage)
    
    return 1 - f_gate

In [33]:
start = time.time()

fid_in = 1 - get_infidelity([ 301.68856998890726,  10])

end = time.time()

print("Computation time = {} s ".format(end - start))

Computation time = 110.35465955734253 s 


In [44]:
start = time.time()

res = minimize(get_infidelity, params_in_new, method="Nelder-Mead", options={"maxiter": 40})

end = time.time()

print("Computation time = {} s".format(end - start))

Computation time = 8663.30581998825 s


In [45]:
res

 final_simplex: (array([[284.35845254,   1.27235587],
       [284.35836966,   1.27259378],
       [284.35794613,   1.27233032]]), array([0.01128775, 0.01128775, 0.01128775]))
           fun: 0.011287745179234365
       message: 'Maximum number of iterations has been exceeded.'
          nfev: 81
           nit: 40
        status: 2
       success: False
             x: array([284.35845254,   1.27235587])

In [46]:
1 - res["fun"]

0.9887122548207656

In [43]:
params_in_new = [282.71518727, 1.45507812]

In [41]:
fid_in

0.9876638688911411

In [42]:
res["x"]

array([282.71518727,   1.45507812])